<a href="https://colab.research.google.com/github/kholodkandeel/Covid-19-EDA/blob/main/BMB_BI_dev_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=f7629de8481b076348eb53b1d965cb6106afa2d62c6ce5122eddc74a636c39ea
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [2]:
# Load important libraries
import pandas as pd
import pandasql as ps

In [3]:
# Load the Excel file
file_path = '/content/Telco Data.xlsx'
xls = pd.ExcelFile(file_path)

# Load the data from both sheets
data_df = pd.read_excel(xls, sheet_name='Telco Data')
code_mapping_df = pd.read_excel(xls, sheet_name='Code Mapping')

In [4]:
data_df.head(5)

,CustomerID,Country Code,City Code,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service and Information,Multiple Lines,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,3,13,male,No,No,No,2,"Yes, +96598533916",No,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,2,f,No,No,Yes,2,"Yes, +961 848715",No,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,5,19,female,No,No,Yes,8,"Yes, +966161584432",Yes,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,1,f,No,Yes,Yes,28,"Yes, +961 977226",Yes,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,4,18,m,No,No,Yes,49,"Yes, +971435222994",Yes,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [5]:
# Merge data with the code mapping to add country and city names
data_df_merged = pd.merge(
    data_df,
    code_mapping_df,
    how='left',
    left_on=['Country Code', 'City Code'],
    right_on=['Country Code', 'City Code']
)

# Drop the old Country Code and City Code columns
data_df_merged = data_df_merged.drop(columns=['Country Code', 'City Code'])

# Display the first few rows to verify
data_df_merged[['Country', 'City']].head()

,Country,City
0,Kuwait,Fahaheel
1,Lebanon,Tripoli
2,KSA,Riyadh
3,Lebanon,Beirut
4,UAE,Ajman


In [6]:
# check for any rows where the splitting may fail
print(data_df_merged['Phone Service and Information'].head(10))  # Check first 10 entries for formatting

# split the column on ',' only if the format looks correct (check that each entry has a comma)
data_df_merged[['Phone Service Registration', 'Phone Number']] = data_df_merged['Phone Service and Information'].str.split(',', n=1, expand=True)

# Drop the old column after verifying the split worked
data_df_merged = data_df_merged.drop(columns=['Phone Service and Information'])

# Verify the split
data_df_merged[['Phone Service Registration', 'Phone Number']].head()

0     Yes, +96598533916
1      Yes, +961 848715
2    Yes, +966161584432
3      Yes, +961 977226
4    Yes, +971435222994
5    Yes, +966733223321
6                    No
7    Yes, +971289293464
8    Yes, +966165486712
9                    No
Name: Phone Service and Information, dtype: object


,Phone Service Registration,Phone Number
0,Yes,+96598533916
1,Yes,+961 848715
2,Yes,+966161584432
3,Yes,+961 977226
4,Yes,+971435222994


In [7]:
# Check the unique values in the "Gender" column to see any irregularities
print(data_df_merged['Gender'].unique())

# Standardize the "Gender" column to only contain "Male" or "Female"
data_df_merged['Gender'] = data_df_merged['Gender'].str.strip().str.lower().map({
    'male': 'Male', 'm': 'Male', 'female': 'Female', 'f': 'Female'
})

# Handle any values that weren't mapped correctly
data_df_merged['Gender'].fillna('Unknown', inplace=True)

# Verify the Gender column
data_df_merged['Gender'].value_counts()


['male' 'f' 'female' 'm' 'Male' 'Female']


<ipython-input-7-ef5b46be2be3>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df_merged['Gender'].fillna('Unknown', inplace=True)


,count
Gender,
Male,3555
Female,3488


In [8]:
# Save the merged data as an Excel file
data_df_merged.to_excel('/content/merged_telco_data.xlsx', index=False)

In [9]:
data_df_merged.head()

,CustomerID,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Multiple Lines,Internet Service,Online Security,Online Backup,...,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason,Country,City,Phone Service Registration,Phone Number
0,3668-QPYBK,Male,No,No,No,2,No,DSL,Yes,Yes,...,108.15,Yes,1,86,3239,Competitor made better offer,Kuwait,Fahaheel,Yes,+96598533916
1,9237-HQITU,Female,No,No,Yes,2,No,Fiber optic,No,No,...,151.65,Yes,1,67,2701,Moved,Lebanon,Tripoli,Yes,+961 848715
2,9305-CDSKC,Female,No,No,Yes,8,Yes,Fiber optic,No,No,...,820.5,Yes,1,86,5372,Moved,KSA,Riyadh,Yes,+966161584432
3,7892-POOKP,Female,No,Yes,Yes,28,Yes,Fiber optic,No,No,...,3046.05,Yes,1,84,5003,Moved,Lebanon,Beirut,Yes,+961 977226
4,0280-XJGEX,Male,No,No,Yes,49,Yes,Fiber optic,No,Yes,...,5036.3,Yes,1,89,5340,Competitor had better devices,UAE,Ajman,Yes,+971435222994


## `Calculate the average monthly charge by country:`

In [10]:
# Query for avg. M by C
query = '''
SELECT Country, AVG("Monthly Charges") as avg_monthly_charge
FROM data_df_merged
GROUP BY Country
'''

average_charge = ps.sqldf(query, locals())
average_charge

,Country,avg_monthly_charge
0,KSA,65.068846
1,Kuwait,64.137034
2,Lebanon,64.843820
3,Qatar,64.987990
4,UAE,64.787116



## `Find the top 5 customers with the highest CLTV`

In [11]:
# Query for top 5 highest CLTV
query_highest = '''
SELECT CustomerID, CLTV
FROM data_df_merged
ORDER BY CLTV DESC
LIMIT 5
'''

highest_cltv = ps.sqldf(query_highest, locals())

highest_cltv

,CustomerID,CLTV
0,7622-FWGEW,6500
1,6024-RUGGH,6499
2,0383-CLDDA,6499
3,2683-JXWQQ,6495
4,8894-JVDCV,6494


## `Find the top 5 customers with the lowest CLTV`

In [12]:
# Query for top 5 lowest CLTV
query_lowest = '''
SELECT CustomerID, CLTV
FROM data_df_merged
ORDER BY CLTV ASC
LIMIT 5
'''
lowest_cltv = ps.sqldf(query_lowest, locals())
lowest_cltv

,CustomerID,CLTV
0,4925-LMHOK,2003
1,0871-URUWO,2003
2,6860-YRJZP,2003
3,7928-VJYAB,2004
4,0247-SLUJI,2004


## `Find customers with monthly charge above 6000`

In [13]:
# Query for customers with monthly charge above 6000
query = '''
SELECT COUNT(*)
FROM data_df_merged
WHERE "Monthly Charges" > 6000
'''

customers_above_6000 = ps.sqldf(query, locals())
customers_above_6000

,COUNT(*)
0,0
